In [1]:
import csv
import re
import numpy as np
from itertools import compress
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import json
from keras.preprocessing.sequence import pad_sequences
import time
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
len_vcb = 366549
path = '/media/rafael/SSD 1Tb/DADOS/Dataset_SP.csv'
batch_size = 256

In [3]:
with open('/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
def consecutive(data, stepsize=1):
    return list(np.split(data, np.where(np.diff(data) != stepsize)[0]+1))

def undersample_data(batch_x, batch_y, Rs):

    undersampled_X_seq = [] # new list of undersampled word samples
    undersampled_Y_seq = [] # new list of undersampled classes samples
    
    for sentence,classe in zip(batch_x, batch_y): # for sentence in dataset

        unique, counts = np.unique(classe, return_counts=True)

        Np = len(classe) - counts[0] # Number of positives tokens
        Nn = counts[0] # Number of negative tokens
        Ns = 0 # Number of selected negative tokens in sentence
        
        array_classe = np.array(classe) # just a convert type from list to numpy array
        array_sentence = np.array(sentence) # just a convert type from list to numpy array

        selected_mask = array_classe > 1 # taking the mask of all positive classes 

        if(Np != 0): # if the setence has at least a single positive tag 
            if(Nn/Np <= Rs): #if the Ratio of Negative tokens is lower then our Ratio: no need for undersampling, select all tokens
                undersampled_X_seq.append(sentence)
                undersampled_Y_seq.append(classe)
            else: 
                while (Ns/Np <= Rs):
                    idx_mask = list(np.where(selected_mask == True)) # select the index where our mask is True
                    idx_consecutives = consecutive(idx_mask[0]) # group a sequence of array
                    # select all in the left first, after all in the right
                    for entity in idx_consecutives:
                        m_left_el = np.min(entity) # most left element
                        try:
                            #selecting the most left element not selected in group
                            if(not selected_mask[m_left_el - 1]): 
                                selected_mask[m_left_el - 1] = True
                                Ns += 1
                        except:
                            pass
                    for entity in idx_consecutives:
                        m_right_el = np.max(entity) # most right element
                        try:
                            #selecting the most left element not selected in group
                            if(not selected_mask[m_right_el + 1]): 
                                selected_mask[m_right_el + 1] = True
                                Ns += 1
                        except:
                            pass
            
                undersampled_Y_seq.append(list(compress(classe, selected_mask.tolist()))) # add the undersampled sample in new list
                undersampled_X_seq.append(list(compress(sentence, selected_mask.tolist()))) # add the undersampled sample in new list

    return undersampled_X_seq, undersampled_Y_seq

In [5]:
def data_generator(batch_size, path, skiprows):
    total = []

    with open(path, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')

        while True:

            batch_x = []
            batch_y = []
            Palavras = []
            Classes = []
            Sentenças = []
            N_batches = 1

            for i, row in enumerate(spamreader):

                if (skiprows != 0):
                    for line in range(skiprows):
                        row.__next__

                indices_virgula = [m.start(0) for m in re.finditer(',', row[0])]
                row = row[0]

                palavra = row[indices_virgula[0]+1: indices_virgula[1]]
                classe = int(row[indices_virgula[1]+1: indices_virgula[2]]) + 1
                sentença = row[indices_virgula[2]+1: indices_virgula[3]]

                if (sentença not in total):
                    total.append(sentença)
                    
                    if (len(total) != 1): # se a lista não é vazia
                        
                        all_is_1 = [True for clas in Classes[0:] if clas != 1]

                        if(len(all_is_1) != 0): # se a sentença possui pelo menos alguma palavra que seja uma classe de interesse

                            batch_x.append(Palavras[0:]) # todas as palavras são uma sentença
                            batch_y.append(Classes[0:])
                            Palavras = [] # logo esvazia o vetor de palavras
                            Classes = []
                        else:
                            Palavras = [] # logo esvazia o vetor de palavras
                            Classes = []
                            del total[-2]
                        

                Palavras.append(palavra)
                Classes.append(classe)
                
                if (len(batch_x) == batch_size):

                    batch_x = tok.texts_to_sequences(batch_x)
                    batch_x, batch_y = undersample_data(batch_x, batch_y, 34)

                    max_len=150

                    batch_x = pad_sequences(maxlen=max_len,
                                    sequences=batch_x,
                                    padding="post",
                                    truncating="post")

                    batch_y = pad_sequences(maxlen=max_len,
                                    sequences=batch_y,
                                    padding="post",
                                    truncating="post")
                    total = []

                    # produz batch de valores (x,y)
                    yield np.array(batch_x), np.array(batch_y)

                    batch_x = []
                    batch_y = []

                    Palavras = []
                    Classes = []
                    Sentenças = []
                    N_batches += 1

                    if (N_batches % 5000 == 0):
                        f = open("saved_line.txt", "w")
                        f.write(f'\n line is: {i}')


In [6]:
gen = data_generator(batch_size, path, 0)

In [7]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, SpatialDropout1D, Bidirectional, Input, Masking

tf.__version__, tf.executing_eagerly()

('2.3.0', True)

In [8]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [9]:
units = 512 # output da LSTM
Emb_dims = 50 # tamanho do vetor de embeddings
Spa_dropout = 0.5 # spacial dropout rate
n_tags = 18 # quantidade de tags

model = Sequential()
model.add(Input(shape=(None,)))
model.add(Embedding(input_dim=len_vcb + 1, output_dim=Emb_dims, mask_zero=True))
model.add(SpatialDropout1D(Spa_dropout))
model.add(Bidirectional(LSTM(units=units, return_sequences=True), merge_mode='sum'))
model.add(Dense(n_tags, activation="softmax"))

In [10]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [11]:
model.compile(optimizer="Adam",
              loss="sparse_categorical_crossentropy",
              metrics=["SparseCategoricalAccuracy",
              recall,
              precision,
              macro_f1])

In [12]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/Callbacks/weights.hdf5',
    save_weights_only=True,
    monitor='macro_f1',
    mode='max',
    save_best_only=True,
    save_freq=5000)

In [13]:
start = time.time()

try:
    with tf.device('/device:GPU:0'):
        model.fit(gen,
                  epochs=1,
                  steps_per_epoch=66102346/batch_size,
                  callbacks=[model_checkpoint_callback])
    
except RuntimeError as e:
    print(e)

print('O modelo levou {} s durante o treino.'.format(time.time()-start))

258213/258212 [==============================] - 141149s 547ms/step - loss: 0.0057 - sparse_categorical_accuracy: 0.9965 - recall: 0.4673 - precision: 0.4950 - macro_f1: 0.4802
O modelo levou 141162.12717437744 s durante o treino.


In [15]:
model.save('SP_TOTAL_1EPOCH.h5')